<h1>FMD Map Creator</h1>
<h2>Authors: Arko Datta and Ayon Chakraborty, SeNSE Lab, IIT Madras</h2>

Offers a step-by-step method to initialize an ∆FMD cache.

<h1>Importing Relevant Packages</h1>

In [ ]:
import sys
sys.path.append("../")

from sionna.rt import load_scene, PlanarArray, Receiver, Transmitter, PathSolver, RadioMap
import mitsuba as mi
import drjit as dr
import gc
from app.configs import *
from scripts.fmd import FMDCalculator
from scripts.utils import render_custom_radiomap
import os
import matplotlib.pyplot as plt
import numpy as np
import json

<h1>Loading the Scene and Material Properties</h1>

If any error should arise, check the cfg_mesh_directory variable in configs.py<br>
Post-update, a kernel restart is needed

In [ ]:
#Load scene
scene=None
try:
    files = os.listdir(cfg_mesh_directory)
except FileNotFoundError:
    raise Exception('Directory not found! Check configs.py and put proper cfg_mesh_directory')
xml_file = None
jsonfile = None
for f in files:
    if f.endswith('.xml'):
        xml_file = f
        print(f"XML file found: {cfg_mesh_directory}/{f}")
        scene = load_scene(f"{cfg_mesh_directory}/{f}")
        print("Scene loaded!")
    if f.endswith('.json'):
        jsonfile = f
        print(f"Material Properties JSON file found: {cfg_mesh_directory}/{f}")
if xml_file is None:
    raise Exception('No xml file found! Check configs.py and put proper cfg_mesh_directory')

<h1>Assigning Material Properties</h1>

In [ ]:
#Assign Material Property
with open(f"{cfg_mesh_directory}/{jsonfile}", "r", encoding="utf-8") as f:
    mat_props = json.load(f)
for obj_name in list(mat_props.keys()):
    obj = scene.get(obj_name)
    obj.radio_material = mat_props[obj_name]
    obj.radio_material.color = cfg_color_dict[mat_props[obj_name]]

<h1>Removing Bounding Box (if configured to do so)</h1>

In [ ]:
bounding_box = scene.get(cfg_bounding_box_name)
if cfg_set_bounding_box_transparent:
    scene.edit(remove=bounding_box)

<h1>Defining the Anchors</h1>

<h2>For our example, we consider the anchors as a grid with a fixed height from the ground. To configure for the same, it is recommended to edit the configurations cfg_tx_grid_length and cfg_tx_grid_breadth. The cfg_scene_length and cfg_scene_breadth should be given properly too.</h2>

In [ ]:
#Configure these to configure the grid of anchors. No need to change further code. 
pad_x = (0,3)
pad_y = (0,2)
height = 1.0

#No need to modify from below. We define the grid
xs = np.linspace(0+pad_x[0], cfg_scene_length-pad_x[1], cfg_tx_grid_length)   # some padding given to adjust overflow from scene
ys = np.linspace(0+pad_y[0], cfg_scene_breadth-pad_y[1], cfg_tx_grid_breadth)    # some padding given to adjust overflow from scene
z  = height                     # fixed height
tx_list = [(x, z, y) for y in ys for x in xs]

#We place the anchors as a grid
scene.tx_array = PlanarArray(
        num_rows=1, num_cols=1,
        vertical_spacing=0.5, horizontal_spacing=0.5,
        pattern="tr38901", polarization="V"
    )
i = 0
for transmitter in tx_list:
    scene.remove(f"tx_{i}")
    tx = Transmitter(name=f"tx_{i}", position=mi.Point3f(float(transmitter[0]),float(transmitter[1]),float(transmitter[2])), display_radius=0.5, look_at=[0,0,0])
    scene.add(tx)
    i+=1

#And we visualize it
scene.preview(clip_at=cfg_clip_at,clip_plane_orientation=cfg_clip_plane_orientation)

<h1>Generating the FMD Cache</h1>

<h2>Initializing the FMD Calculator</h2>

In [ ]:
fmd = FMDCalculator(
    arena_size=(cfg_scene_length, cfg_scene_breadth), spacing=0.5,#We assume 0.5m spacing between cell centroids of FMD map
    real_csv_path=cfg_cir_path,
    scene_path=f"{cfg_mesh_directory}/{xml_file}"
)

<h2>Generating the Cache</h2>

In [ ]:
for i, tx in enumerate(tx_list, 1):
    print(f"\nGenerating for TX {i}/{len(tx_list)}...",end="")
    fmd_vals, _ = fmd.update_fmd(tx)
    print("Done")

<h2>Export the cache to a json file.</h2>

In [ ]:
fmd.export_cache_to_json(cfg_fmd_cache_name,pretty=True)

<h2>If you want to import a cache from a json file.</h2>

In [ ]:
fmd.import_cache_from_json(cfg_fmd_cache_name)

<h1>You can visualize FMD for a given transmitter</h1>

In [ ]:
floor = scene.get(cfg_floor_name)
if cfg_set_floor_transparent:
    scene.edit(remove=floor)

In [ ]:
transmitter_num = 2
fmd_map = fmd.get_fmd_map(tx_list[transmitter_num])
tx_position = tx_list[transmitter_num]
for i in range(len(tx_list)):
    scene.remove(f"tx_{i}")
tx = Transmitter(name=f"tx_{transmitter_num}", position=mi.Point3f(float(tx_position[0]),float(tx_position[1]),float(tx_position[2])), display_radius=0.5, look_at=[0,0,0])
scene.add(tx)

rm = render_custom_radiomap(fmd_map, scene, center=[cfg_scene_length/2, 0.1, cfg_scene_breadth/2], size=[cfg_scene_length, cfg_scene_breadth])


scene.preview(resolution=(1920,1080),radio_map=rm,clip_at=cfg_clip_at, clip_plane_orientation=cfg_clip_plane_orientation,rm_metric="rss")